In [ ]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## Steps to build pipeline

1. Tokenize the query
2. Tokenize the streams
3. Tokenize the stream

Run LSTM on query

In [ ]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')
os.listdir('./')

In [ ]:
df1 = pd.read_csv('./2012_aurora_shooting.tsv.gz', sep='\t')
df2 = pd.read_csv('./2012_pakistan_garment_factory_fires.tsv.gz', sep='\t')
df3 = pd.read_csv('./hurricane_sandy.tsv.gz', sep='\t')
df4 = pd.read_csv('./wisconsin_sikh_temple_shooting.tsv.gz', sep='\t')

In [ ]:
# Title is the title of the article
print df1[df1['stream id']=='1342767285-e9b3e7dc6a9e31c8b2301848ab082117'].title[1]
# Each text is a stream (i.e., a sentence from the article)
print df1[df1['stream id']=='1342767285-e9b3e7dc6a9e31c8b2301848ab082117'].text[1]

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3.head()

In [ ]:
df4.head()

In [ ]:
os.listdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/trec-2013-data/')

In [ ]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/trec-2013-data/')

In [ ]:
df5 = pd.read_csv('./nuggets.tsv.gz', sep='\t')
df6 = pd.read_csv('./matches.tsv.gz', sep='\t')
df7 = pd.read_csv('./updates.tsv.gz', sep='\t')

In [ ]:
df5.head() # Nuggets

In [ ]:
df6.head() # Matches

In [ ]:
df7.head() # Updates

In [ ]:
df7['query_id'].nunique()

In [ ]:
f = open('./trec2013-ts-topics-test.xml', 'rb')

out = f.readlines()
ox = BeautifulSoup(''.join(out),'lxml').contents[1]

In [ ]:
i.findAll('title')[0].text, ',',i.findAll('query')[0].text

In [ ]:
print "Wiki article, Query"
qs = []
c = 0
for i in ox.findAll('event'):
    qs.append(i.findAll('query')[0].text)
    print c, i.findAll('title')[0].text, ',',i.findAll('query')[0].text
    c+=1


# Only have ten events, so we should build the code to be more general

## This is the stream data for Aurora -- all the same query

In [ ]:
# This is the aurora shooting, query #2 in the xml
df1.head()

## these are the nuggets for all of the trek data

In [ ]:
df5.shape

In [ ]:
df5.head()

In [ ]:
df5[df5['nugget_id']=='VMTS13.01.077'] # Nuggets

Index in df6 (matches) is concatenation of (queryid, streamid, and sentence)

In [ ]:
df6.shape

In [ ]:
df6.head()

## These are the matches of the nuggets

In [ ]:
df7.head()

# This section tokenizes each dataset into a list of numeric indices

In [8]:
import os
import re
import sys
import pickle
import csv
import pandas as pd
from gensim import corpora
from collections import defaultdict    

def BuildIndexFiles(infile_list):
    """
    :type  infilename: str
    :param infilename: Fulll name of input file 

    :type  outfilename: str
    :param outfilename: Export filename, without the '.csv'
    """
    reload(sys)
    sys.setdefaultencoding('utf-8')
    all_tokens = []
    frequency = defaultdict(int)
    for idx, infilename in enumerate(infile_list):
        print('Loading %s %i of %i' % (infilename, idx, len(infile_list)) )

        df = pd.read_csv(infilename, sep='\t', encoding='latin-1')
        df['text'] = df['text'].str.replace('[^A-Za-z0-9]+', ' ').str.strip()
        texts = [ t.split(" ") for t in df['text'] ]

        for text in texts:
            for token in text:
                frequency[token] += 1
        texts = [ [token for token in text ]  for text in texts]
        # Collecting all the list of tokens
        all_tokens.append(texts)

    texts = sum(all_tokens, [])
    # Getting the dictionary with token info
    dictionary = corpora.Dictionary(texts)
    # Mapping to numeric list
    word2idx = dictionary.token2id
    dictionary.id2token = {v:k for k,v in dictionary.token2id.items()}
    idx2word = dictionary.id2token
    
    # Exporting the dictionaries
    print("Exporting word to index and dictionary to word indices")
    output = open('./0-output/LSTMDQN_Dic_token2id.pkl', 'ab+')
    pickle.dump(word2idx, output)
    output.close()

    output = open('./0-output/LSTMDQN_Dic_id2token.pkl', 'ab+')
    pickle.dump(idx2word, output)
    output.close()
    
    # Merging the dictionaries toa pandas data frame with summary info
    odf0 = pd.DataFrame.from_dict(dictionary.dfs, orient='index').reset_index()
    odf1 = pd.DataFrame.from_dict(word2idx, orient='index').reset_index()
    odf0.columns = ['id', 'frequency']
    odf1.columns = ['token', 'id']
    odf = pd.merge(left=odf0, right=odf1, on='id')
    odf = odf[['id','token', 'frequency']]
    odf.to_csv('./0-output/total_corpus_smry.csv')
    
    return dictionary
    
def TokenizeData(infile_list, outfile_list, word2idx):
    for idx, (infilename, outfilename) in enumerate(zip(infile_list, outfile_list)):
        df = pd.read_csv(infilename, sep='\t', encoding='latin-1')
        df['text'] = df['text'].str.replace('[^A-Za-z0-9]+', ' ').str.strip()
        texts = [ t.split(" ") for t in df['text'] ]

        frequency = defaultdict(int)
        for text in texts:
            for token in text:
                frequency[token] += 1
        texts = [ [token for token in text ]  for text in texts]
        
        text_numindex = [ [word2idx[i] for i in t] for t in texts]
        # Exporting files
        print('...file export to %s' % outfilename)
        with open(outfilename+'_numtext.csv', 'wb') as csvfile:
            data = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            data.writerow(['Text'])
            data.writerows(text_numindex)

        csvfile.close()
    print('...Exporting of tokenized data complete')

In [2]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')

infilelist = [
            './2012_aurora_shooting.tsv.gz', 
            './2012_pakistan_garment_factory_fires.tsv.gz',
            './hurricane_sandy.tsv.gz',
            './wisconsin_sikh_temple_shooting.tsv.gz'
]
outfilelist = [
            './0-output/2012_aurora_shooting',
            './0-output/2012_pakistan_garment_factory_fires',
            './0-output/hurricane_sandy',
            './0-output/wisconsin_sikh_temple_shooting'
    
]

In [3]:
mycorpora = BuildIndexFiles(infilelist)

In [9]:
TokenizeData(infilelist, outfilelist, word2idx=mycorpora.token2id)